In [1]:
from IPython.display import Image

- DAG: Directed Acyclic Graph
    - 有向无环图；
        - 环：means cycle back；
    - formalized workflow for transitioning data
    - 一个节点可以有多个出边（如下图的节点 `a`），即有多条 outgoing 的路径；
        - 自然一个节点也可以有多个入边，如下图的节点 `d`；
    - DAG的主要关注点并不是任务内部的运作机制，而是它们应该如何执行，包括执行的顺序、重试逻辑、超时以及其他操作方面的问题。这种抽象使得创建复杂的工作流变得容易管理和监控。
        - https://www.51cto.com/article/781996.html
- langgraph 比着 DAG 很大的一个改进就是存在循环；

In [4]:
Image(url='./imgs/dag.png', width=300)

### prompt template

- https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html

In [3]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

### messages

https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/

- All messages have a role and a content property.
    - additional_kwargs
        - function_call
- message types
    - HumanMessage
    - AIMessage: role, assistant
        - a message from the model.
        - additional_kwargs (tool_calls)
    - FunctionMessage: role, tool
        - represents the result of a function call.
        - role, content, name
    - ToolMessage: role, tool
        - the result of a tool call.
        - his is distinct from a FunctionMessage in order to match OpenAI's `function` and `tool` message types. 
        - role, content, tool_call_id

In [3]:
# https://platform.openai.com/docs/guides/function-calling
Image(url='https://cdn.openai.com/API/docs/images/function-calling-diagram.png', width=500)

### lcel

In [1]:
from dotenv import load_dotenv
assert load_dotenv()

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

- StrOutputParser(): 作用的对象是 Message

In [4]:
chain.invoke({'topic': 'pig'})

'Why did the pig go to the casino? Because he heard they had a lot of "squeal" machines!'

In [9]:
chain.first.invoke({'topic': 'pig'})

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about pig', additional_kwargs={}, response_metadata={})])

In [5]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [6]:
chain.invoke({'topic': 'pig'})

'Why did the pig go to the casino? He heard they had a lot of bacon!'

In [7]:
chain.first.invoke({'topic': 'pig'})

StringPromptValue(text='tell me a short joke about pig')

In [10]:
prompt = PromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | (lambda x: x.text) | model | output_parser

In [11]:
chain.invoke({'topic': 'pig'})

'Why did the pig go to the kitchen? Because he felt like bacon!'

#### members

In [7]:
chain.first

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a short joke about {topic}'))])

In [8]:
chain.first.invoke({'topic': 'apple'})

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about apple')])

In [9]:
chain.first.invoke({'topic': 'apple'}).to_messages()

[HumanMessage(content='tell me a short joke about apple')]

In [10]:
chain.middle

[ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fb4fb882330>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fb4fb883c80>, openai_api_key=SecretStr('**********'), openai_proxy='')]

In [11]:
res = chain.middle[0].invoke(chain.first.invoke({'topic': 'math'}))

In [12]:
res

AIMessage(content="Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else.", response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 14, 'total_tokens': 36, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9823590f-d239-4bce-bb18-5075dccc56ee-0', usage_metadata={'input_tokens': 14, 'output_tokens': 22, 'total_tokens': 36})

In [13]:
chain.last

StrOutputParser()

In [14]:
chain.last.invoke(res)

"Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else."

In [15]:
chain.last.invoke(chain.first.invoke({'topic': 'apple'}).messages[0])

'tell me a short joke about apple'

#### RAG chain 

```mermaid
graph LR
    Question --> RunnableParallel
    RunnableParallel --> |Question| Retriever
    RunnableParallel --> |Question| RunnablePassThrough
    Retriever --> |context-retrieved docs| PromptTemplate
    RunnablePassThrough --> |question=Question| PromptTemplate
    PromptTemplate --> |PromptValue| ChatModel
    ChatModel --> |ChatMessage| StrOutputParser
    StrOutputParser --> |String| Result
```

In [16]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# two branch
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

/home/whaow/anaconda3/envs/agent/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Harrison worked at Kensho.'

In [17]:
setup_and_retrieval.invoke("where did harrison work?")

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

In [18]:
prompt.invoke(setup_and_retrieval.invoke("where did harrison work?"))

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(page_content='harrison worked at kensho'), Document(page_content='bears like to eat honey')]\n\nQuestion: where did harrison work?\n")])

In [19]:
model.invoke(prompt.invoke(setup_and_retrieval.invoke("where did harrison work?")))

AIMessage(content='Harrison worked at Kensho.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 50, 'total_tokens': 57, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-398933d0-e643-402f-9c41-ee440b66f642-0', usage_metadata={'input_tokens': 50, 'output_tokens': 7, 'total_tokens': 57})